<H1 align="center">  Cleaning and Transformation </H1>

---

# The data

In this excercise I'll be using data from the [US Public Food Assistance 1 - WIC](https://www.kaggle.com/datasets/jpmiller/publicassistance) Kaggle database that gives us insights on WIC expenditure accross different states and government organizations and poverty statistics collected by the U.S. Census Bureau.

The tables presented in this workbook correspond to the fiscal year (October to September) 2016 since this is the latest data presented in the original database. We will work with only 2 tables: ` Total_Number_of_Participants` that gives us the number of enrolled in WIC accross different states and government organizations and ` est16us`  that is poverty statistics.

# The tools

I'll use SQLite3 given the popularity and API integration it has with Jupyter notebooks.

# The objective

The final products of this excercise should be a `Participants` table that only counts WIC agencies at the state level and a `census2016` table that includes only the needed columns and has accurate and contingent data. Both tables should not include duplicates and should have the correct data types for their purposes.

# The excercise

In [1]:
!git clone https://github.com/arboles95/wic.git

Cloning into 'wic'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 1), reused 6 (delta 1), pack-reused 0
Receiving objects: 100% (6/6), 63.17 KiB | 15.79 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [6]:
import pandas as pd
import numpy as np
import sqlite3

In [8]:
path = '/content/wic/wic.db'

conn = sqlite3.connect(path)

cursor = conn.cursor()

tables = pd.read_sql_query("""SELECT *
                              FROM sqlite_master
                              WHERE type='table';""", conn)
print(tables)

    type                          name                      tbl_name  \
0  table  Total_Number_of_Participants  Total_Number_of_Participants   
1  table                       est16us                       est16us   

   rootpage                                                sql  
0        41  CREATE TABLE "Total_Number_of_Participants" (\...  
1        54  CREATE TABLE "est16us" (\n\t"GeographicInforma...  


First we'll check the data types for the participants table

In [9]:
query = """PRAGMA table_info(Total_Number_of_Participants)"""

cursor.execute(query)

for row in cursor.fetchall():
    print(row)

(0, 'StateAgencyorIndianTribalOrganization', 'TEXT', 0, None, 0)
(1, '2015-10-0100:00:00', 'REAL', 0, None, 0)
(2, '2015-11-0100:00:00', 'REAL', 0, None, 0)
(3, '2015-12-0100:00:00', 'REAL', 0, None, 0)
(4, '2016-01-0100:00:00', 'REAL', 0, None, 0)
(5, '2016-02-0100:00:00', 'REAL', 0, None, 0)
(6, '2016-03-0100:00:00', 'REAL', 0, None, 0)
(7, '2016-04-0100:00:00', 'REAL', 0, None, 0)
(8, '2016-05-0100:00:00', 'REAL', 0, None, 0)
(9, '2016-06-0100:00:00', 'REAL', 0, None, 0)
(10, '2016-07-0100:00:00', 'REAL', 0, None, 0)
(11, '2016-08-0100:00:00', 'REAL', 0, None, 0)
(12, '2016-09-0100:00:00', 'REAL', 0, None, 0)
(13, 'AverageParticipation', 'INTEGER', 0, None, 0)


The columns counting participants are REAL (floats) data types and average participations is an INTEGER type.

These should be the other way around.

In [10]:
query1 = """CREATE TEMPORARY TABLE IF NOT EXISTS Participants AS
            SELECT
                StateAgencyorIndianTribalOrganization AS place,
                CAST("2015-10-0100:00:00" AS INTEGER) AS "2015-10",
                CAST("2015-11-0100:00:00" AS INTEGER) AS "2015-11",
                CAST("2015-12-0100:00:00" AS INTEGER) AS "2015-12",
                CAST("2016-01-0100:00:00" AS INTEGER) AS "2016-01",
                CAST("2016-02-0100:00:00" AS INTEGER) AS "2016-02",
                CAST("2016-03-0100:00:00" AS INTEGER) AS "2016-03",
                CAST("2016-04-0100:00:00" AS INTEGER) AS "2016-04",
                CAST("2016-05-0100:00:00" AS INTEGER) AS "2016-05",
                CAST("2016-06-0100:00:00" AS INTEGER) AS "2016-06",
                CAST("2016-07-0100:00:00" AS INTEGER) AS "2016-07",
                CAST("2016-08-0100:00:00" AS INTEGER) AS "2016-08",
                CAST("2016-09-0100:00:00" AS INTEGER) AS "2016-09",
                CAST(AverageParticipation AS REAL) AS AvgParticipation
            FROM Total_Number_of_Participants
"""

cursor.execute(query1)

query2 = """SELECT *
            FROM Participants
            ORDER BY place DESC
            LIMIT 15"""

cursor.execute(query2)

for row in cursor.fetchall():
    print(row)

('Zuni Pueblo, NM', 684, 625, 662, 635, 661, 626, 670, 615, 660, 634, 667, 584, 643.5833333333334)
('Wyoming', 10316, 10220, 10216, 10233, 10201, 10275, 10238, 10195, 10214, 10003, 10056, 10082, 10187.416666666666)
('Wisconsin', 103471, 102042, 100895, 100545, 99349, 99671, 99373, 99286, 99290, 98519, 99855, 99527, 100151.91666666669)
('Winnebago Tribe, NE', 292, 261, 249, 272, 227, 265, 244, 259, 257, 283, 298, 253, 263.3333333333333)
('Wichita, Caddo & Delaware (WCD), OK', 3955, 3910, 3893, 4019, 4010, 3994, 3965, 4029, 4101, 4014, 4049, 3910, 3987.4166666666665)
('West Virginia', 41477, 40837, 40668, 40318, 40055, 40457, 40223, 39957, 39904, 39788, 40461, 40546, 40390.916666666664)
('Washington', 174838, 170214, 170231, 170225, 168323, 170830, 168599, 167354, 168203, 164902, 166689, 166122, 168877.5)
('Virginia', 139899, 136173, 134223, 131145, 130096, 131866, 131315, 131515, 131180, 130095, 131130, 130211, 132404.0)
('Virgin Islands', 4322, 4313, 4240, 4258, 4268, 4176, 4151, 4194,

Texas appears twice here, with one record having all nulls. We'll have to remove this.

In [11]:
query1 = """DELETE FROM Participants
            WHERE place = 'Texas' AND "2015-10" IS NULL"""

cursor.execute(query1)

query2 = """SELECT *
            FROM Participants
            WHERE place = 'Texas' """

cursor.execute(query2)

for row in cursor.fetchall():
    print(row)

('Texas', 882897, 875264, 862979, 859663, 858671, 858889, 852957, 852304, 858258, 847738, 855542, 852671, 859819.4166666666)


We can proceed with filtering for dependencies at the state level.

First we'll query the names of states from the est16us table. (The est16us table also needs cleaning and transformation for column names but we'll get to that later.)

In [13]:
query1 = """CREATE TEMPORARY TABLE IF NOT EXISTS States AS
            SELECT "Unnamed:2" AS Names
            FROM est16us
            WHERE "Unnamed:1" NOT IN ("US", "DC", "Postal Code")"""

cursor.execute(query1)

query2 = """SELECT *
            FROM States """

cursor.execute(query2)

for row in cursor.fetchall():
    print(row)

('Alabama',)
('Alaska',)
('Arizona',)
('Arkansas',)
('California',)
('Colorado',)
('Connecticut',)
('Delaware',)
('Florida',)
('Georgia',)
('Hawaii',)
('Idaho',)
('Illinois',)
('Indiana',)
('Iowa',)
('Kansas',)
('Kentucky',)
('Louisiana',)
('Maine',)
('Maryland',)
('Massachusetts',)
('Michigan',)
('Minnesota',)
('Mississippi',)
('Missouri',)
('Montana',)
('Nebraska',)
('Nevada',)
('New Hampshire',)
('New Jersey',)
('New Mexico',)
('New York',)
('North Carolina',)
('North Dakota',)
('Ohio',)
('Oklahoma',)
('Oregon',)
('Pennsylvania',)
('Rhode Island',)
('South Carolina',)
('South Dakota',)
('Tennessee',)
('Texas',)
('Utah',)
('Vermont',)
('Virginia',)
('Washington',)
('West Virginia',)
('Wisconsin',)
('Wyoming',)


Now that we have the States table we can proceed to filter the Participants table.

In [14]:
query1 = """DELETE FROM Participants
            WHERE place NOT IN States"""

cursor.execute(query1)

query2 = """SELECT *
            FROM Participants
            LIMIT 10"""

cursor.execute(query2)

for row in cursor.fetchall():
    print(row)

('Connecticut', 50960, 49600, 49922, 49803, 48351, 49374, 48162, 48920, 48908, 48221, 48842, 48798, 49155.083333333336)
('Maine', 21180, 21099, 20952, 21006, 20795, 20866, 20628, 20488, 20310, 20139, 20466, 20286, 20684.583333333332)
('Massachusetts', 114142, 115972, 114682, 112913, 113624, 113843, 113273, 116040, 111516, 114669, 116258, 116091, 114418.58333333331)
('New Hampshire', 14689, 14301, 14182, 14166, 14059, 14263, 13916, 14159, 14139, 13945, 13888, 13750, 14121.416666666666)
('New York', 470638, 467100, 462290, 459356, 457118, 458835, 457372, 458136, 456905, 451073, 454833, 453304, 458913.3333333333)
('Rhode Island', 20934, 20701, 20694, 20446, 19853, 20420, 20150, 20443, 20434, 19836, 20162, 20430, 20375.25)
('Vermont', 13422, 13384, 13217, 13190, 12876, 12686, 12799, 12600, 12472, 12251, 12134, 12036, 12755.583333333334)
('Delaware', 18683, 18346, 18276, 18458, 18210, 18226, 18249, 18004, 17586, 17503, 17797, 17831, 18097.416666666668)
('Maryland', 143142, 141468, 140192, 1

It's probably best to update the AvgParticipation column given all the cleaning we've had to do.

In [15]:
query1 = """UPDATE Participants
            SET AvgParticipation = (
            SELECT ROUND(AVG(("2015-10" + "2015-11" + "2015-12" + "2016-01" + "2016-02" + "2016-03" +
            "2016-04" + "2016-05" + "2016-06" + "2016-07" + "2016-08" + "2016-09")/12), 2)
            FROM Participants as p
            WHERE p.place = Participants.place)"""

cursor.execute(query1)

query2 = """SELECT *
            FROM Participants
            LIMIT 10"""

cursor.execute(query2)

for row in cursor.fetchall():
    print(row)

('Connecticut', 50960, 49600, 49922, 49803, 48351, 49374, 48162, 48920, 48908, 48221, 48842, 48798, 49155.0)
('Maine', 21180, 21099, 20952, 21006, 20795, 20866, 20628, 20488, 20310, 20139, 20466, 20286, 20684.0)
('Massachusetts', 114142, 115972, 114682, 112913, 113624, 113843, 113273, 116040, 111516, 114669, 116258, 116091, 114418.0)
('New Hampshire', 14689, 14301, 14182, 14166, 14059, 14263, 13916, 14159, 14139, 13945, 13888, 13750, 14121.0)
('New York', 470638, 467100, 462290, 459356, 457118, 458835, 457372, 458136, 456905, 451073, 454833, 453304, 458913.0)
('Rhode Island', 20934, 20701, 20694, 20446, 19853, 20420, 20150, 20443, 20434, 19836, 20162, 20430, 20375.0)
('Vermont', 13422, 13384, 13217, 13190, 12876, 12686, 12799, 12600, 12472, 12251, 12134, 12036, 12755.0)
('Delaware', 18683, 18346, 18276, 18458, 18210, 18226, 18249, 18004, 17586, 17503, 17797, 17831, 18097.0)
('Maryland', 143142, 141468, 140192, 138100, 138914, 139343, 139662, 139316, 139982, 137989, 138926, 137877, 1395

Clean and tidy.

We can proceed with cleaning the est16us table, which is the table with poverty indicators from the US Census Bureau for 2016.


In [ ]:
query = """PRAGMA table_info(est16us)"""

cursor.execute(query)

for row in cursor.fetchall():
    print(row)

This table has 30 columns, most of which are unnamed. The formatting on this table was erronous but it at least gave us the first row so we can consult what information these columns are storing.

We see that most of the columns are poverty estimates and percentages accross age groups and ther 90% CI Upper and Lower boundaries.

We'll have to seperate the columns with the CI boundearies from our replacement table. While we're at it we'll give the columns proper names. The types will also have to be corrected when being queried for the new table.

In [16]:
query1 = """CREATE TEMPORARY TABLE IF NOT EXISTS census2016 AS
	SELECT
	CAST(GeographicInformation AS INTEGER) AS State_Code,
	"Unnamed:1" AS Postal_Code,
	"Unnamed:2" AS Name,
	CAST(AllAges AS INTEGER) AS Pov_All_Ages_Est,
	CAST("Unnamed:6" AS REAL) AS Pov_All_Ages_Perc,
	CAST("Age0-17" AS INTEGER) AS Pov_0_17_Est,
	CAST("Unnamed:12" AS REAL) AS Pov_0_17_Perc,
	CAST("Age5-17inFamilies" AS INTEGER) AS Pov_5_17_Fam_Est,
	CAST("Unnamed:18" AS REAL) AS Pov_5_17_Fam_Perc,
	CAST(MedianHouseholdIncome AS INTEGER) AS Med_Household_Income,
	CAST("Age0-4" AS INTEGER) AS Pov_0_4_Est,
	CAST("Unnamed:27" AS REAL) AS Pov_0_4_Perc
	FROM est16us"""

cursor.execute(query1)

query2 = """DELETE FROM census2016
            WHERE Name NOT IN States"""

cursor.execute(query2)

query3 = """SELECT *
            FROM census2016
            LIMIT 10"""

cursor.execute(query3)

for row in cursor.fetchall():
    print(row)

(1, 'AL', 'Alabama', 814197, 17.2, 267674, 24.7, 185889, 23.5, 46309, 78675, 27.4)
(2, 'AK', 'Alaska', 71916, 9.9, 24897, 13.6, 16061, 12.3, 76144, 7919, 15.1)
(4, 'AZ', 'Arizona', 1107153, 16.4, 377445, 23.6, 263614, 22.6, 53481, 106817, 24.9)
(5, 'AR', 'Arkansas', 497388, 17.2, 165724, 24.0, 112376, 22.3, 44406, 50341, 27.3)
(6, 'CA', 'California', 5527621, 14.4, 1782764, 19.9, 1242780, 19.2, 67715, 502432, 20.7)
(8, 'CO', 'Colorado', 598378, 11.0, 166581, 13.4, 115917, 12.8, 65718, 46111, 13.9)
(9, 'CT', 'Connecticut', 342171, 9.9, 97462, 13.1, 67644, 12.1, 73380, 27362, 15.2)
(10, 'DE', 'Delaware', 109592, 11.8, 35702, 17.7, 23942, 16.3, 62112, 10824, 20.1)
(12, 'FL', 'Florida', 2986237, 14.8, 869892, 21.3, 594181, 20.1, 50857, 259244, 23.5)
(13, 'GA', 'Georgia', 1612368, 16.1, 572102, 23.1, 390690, 21.5, 53468, 172046, 26.6)


# Conclusion

The importance of clean and tidy data is paramount to analysis. The data I've worked with in this excercise also signaled the need to check and cast data types in the tables we'll be working with since they're not always clear at first sight. I've also had to transform the data given duplicates that wouldn't have been detected if not actively searched for. Some columns were also not relevant to the task at hand and that called for seperating them from the final table, to facilitate analysis.